In [1]:
from langchain.chains import LLMMathChain
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain import hub
from pydantic import BaseModel, Field
from langchain.tools import BaseTool
from typing import Type
from hellocomputer.config import settings
from hellocomputer.models import AvailableModels

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-tools-agent")

llm = ChatOpenAI(
    base_url=settings.llm_base_url,
    api_key=settings.llm_api_key,
    model=AvailableModels.firefunction_2,
    temperature=0.5,
    max_tokens=256,
)

math_llm = ChatOpenAI(
    base_url=settings.llm_base_url,
    api_key=settings.llm_api_key,
    model=AvailableModels.mixtral_8x7b,
    temperature=0.5,
)

In [2]:
class CalculatorInput(BaseModel):
    query: str = Field(description="should be a math expression")

class CustomCalculatorTool(BaseTool):
    name: str = "Calculator"
    description: str = "Tool to evaluate mathemetical expressions"
    args_schema: Type[BaseModel] = CalculatorInput

    def _run(self, query: str) -> str:
        """Use the tool."""
        return LLMMathChain.from_llm(llm=math_llm, verbose=True).invoke(query)

    async def _arun(self, query: str) -> str:
        """Use the tool asynchronously."""
        return LLMMathChain.from_llm(llm=math_llm, verbose=True).ainvoke(query)

tools = [
  CustomCalculatorTool()
]

agent = create_openai_tools_agent(llm, tools, prompt)

agent = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [3]:
print(agent.invoke({"input": "What is the capital of USA?"}))



> Entering new AgentExecutor chain...
The capital of USA is Washington, D.C.

> Finished chain.
{'input': 'What is the capital of USA?', 'output': 'The capital of USA is Washington, D.C.'}


In [4]:
print(agent.invoke({"input": "What is 100 divided by 25?"}))



> Entering new AgentExecutor chain...

Invoking: `Calculator` with `{'query': '100/25'}`




> Entering new LLMMathChain chain...
100/25```text
100 / 25
```
...numexpr.evaluate("100 / 25")...

Answer: 4.0
> Finished chain.
{'question': '100/25', 'answer': 'Answer: 4.0'}The answer is 4.0.

> Finished chain.
{'input': 'What is 100 divided by 25?', 'output': 'The answer is 4.0.'}


# Now let's modify this code and make it a Graph

In [6]:
from langgraph.prebuilt import ToolNode
from langchain_core.messages import AIMessage

tool_node = ToolNode(tools=tools)

In [7]:
# Call the tools node manually

message_with_single_tool_call = AIMessage(
    content="",
    tool_calls=[
        {
            "name": "Calculator",
            "args": {"query": "50 / 2"},
            "id": "tool_call_id",
            "type": "tool_call",
        }
    ],
)

tool_node.invoke({"messages": [message_with_single_tool_call]})



> Entering new LLMMathChain chain...
50 / 2```text
50 / 2
```
...numexpr.evaluate("50 / 2")...

Answer: 25.0
> Finished chain.


{'messages': [ToolMessage(content='{"question": "50 / 2", "answer": "Answer: 25.0"}', name='Calculator', tool_call_id='tool_call_id')]}

In [8]:
# Bind the tools to the agent so it knows what to call

agent = ChatOpenAI(
    base_url="https://api.fireworks.ai/inference/v1",
    api_key="bGWp5ErQNI7rP8GOcGBJmyC5QMV7z8UdBpLAseTaxhbAk6u1",
    model="accounts/fireworks/models/firefunction-v2",
    temperature=0.5,
    max_tokens=256,
).bind_tools(tools)

In [9]:
agent.invoke("234/7").tool_calls

[{'name': 'Calculator',
  'args': {'query': '234/7'},
  'id': 'call_mP5fctn8N6vilM1Yewb5QxRY',
  'type': 'tool_call'}]

In [10]:
from typing import Literal

from langgraph.graph import StateGraph, MessagesState


def should_continue(state: MessagesState) -> Literal["tools", "__end__"]:
    messages = state["messages"]
    last_message = messages[-1]
    if last_message.tool_calls:
        return "tools"
    return "__end__"


def call_model(state: MessagesState):
    messages = state["messages"]
    response = agent.invoke(messages)
    return {"messages": [response]}


workflow = StateGraph(MessagesState)

# Define the two nodes we will cycle between
workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)

workflow.add_edge("__start__", "agent")
workflow.add_conditional_edges(
    "agent",
    should_continue,
)
workflow.add_edge("tools", "agent")

app = workflow.compile()

In [11]:
for chunk in app.stream(
    {"messages": [("human", "What's twelve times twelve?")]}, stream_mode="values"
):
    chunk["messages"][-1].pretty_print()


================================ Human Message =================================

What's twelve times twelve?
================================== Ai Message ==================================
Tool Calls:
  Calculator (call_dEdudo8txWMpLKMek070TIWd)
 Call ID: call_dEdudo8txWMpLKMek070TIWd
  Args:
    query: 12 * 12


> Entering new LLMMathChain chain...
12 * 12```text
12 * 12
```
...numexpr.evaluate("12 * 12")...

Answer: 144
> Finished chain.
================================= Tool Message =================================
Name: Calculator

{"question": "12 * 12", "answer": "Answer: 144"}
================================== Ai Message ==================================

The answer is 144.
